In [0]:
#!import "../DataModel/DataStructure"

In [0]:
public record CashflowIdentity{
    public string DataNode {get; set;}
    public string AmountType {get; set;}
    public int? AccidentYear {get; set;}
    public Type Type {get; set;}
}

# Storage

In [0]:
public class CashflowStorage
{   
    private readonly IDataSource querySource; 
    private readonly IWorkspace workspace;
    private readonly Systemorph.Vertex.Hierarchies.IHierarchicalDimensionCache hierarchyCache;
    private readonly ImportArgs args;

    public Cashflow[] CashflowsByIdentity { get; private set; }
    public CashflowIdentity[] Identities {get; set;}

    // Constructor
    public CashflowStorage(ImportArgs args, IDataSource querySource, IWorkspace workspace)
    {
        this.querySource = querySource;
        this.workspace = workspace;
        hierarchyCache = workspace.ToHierarchicalDimensionCache();
        this.args = args;
    }
    
    // Initialize
    public async Task InitializeAsync()
    {
        CashflowsByIdentity = await workspace.Query<Cashflow>().ToArrayAsync();
        // Identities
        Identities = CashflowsByIdentity.Select(x => new CashflowIdentity() 
            { 
                DataNode = x.DataNode,
                AmountType = x.AmountType,
                AccidentYear = x.AccidentYear,
                Type = x.GetType(),
            }
        ).Distinct().ToArray();
    }
   
    // Getters
    public IEnumerable<Cashflow> GetCashflow<T>(CashflowIdentity identity) where T : Cashflow =>
        CashflowsByIdentity.Where(x => x.GetType() == typeof(T) && x.DataNode == identity.DataNode && x.AmountType == identity.AmountType && x.AccidentYear == identity.AccidentYear);
}